# MLB.com stats crawler
https://github.com/whyhugo/Data-Analysis-in-Sports
> Please input the URL which can get from https://www.mlb.com/stats

In [1]:
import requests as rq
import pandas as pd
from bs4 import BeautifulSoup as soup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from lxml import etree
import time

In [2]:

chrome_options = Options()
chrome_options.add_argument('--headless')



url = input('URL:')
driver = webdriver.Chrome('/programing/swiftx/chromedriver-win64/chromedriver.exe', options=chrome_options)
html = driver.get(url)
sp = soup(driver.page_source, 'html.parser')

URL: https://www.mlb.com/stats/arizona-diamondbacks?playerPool=ALL


C:\Users\user\AppData\Local\Temp\ipykernel_28516\4031441934.py:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('/programing/swiftx/chromedriver-win64/chromedriver.exe', options=chrome_options)


get player index
```python
index_list = []
index = sp.find_all(class_ = 'index-3cdMSKi7')
for i in range(0, len(index), 2):
    index_list.append(index[i].text)
```

In [3]:
# get player name
def get_player_name():
    player_list = []
    player = sp.find_all(class_ = 'full-3fV3c9pF')
    for i in range(0, len(player), 2):
        player_list.append(player[i].text + ' ' + player[i+1].text)
    return player_list

In [4]:
# get player's position
def get_player_position():
    position_list = []
    position = sp.find_all(class_ = 'position-28TbwVOg')
    for i in position:
        position_list.append(i.text)
    return position_list

In [5]:
def get_column_name():
    click_standard_button()
    sp = soup(driver.page_source, 'html.parser')
    
    data_column_name = ['PLAYER', 'POSITION', 'TEAM']
    temp = sp.find_all(class_ = 'bui-text cellheader bui-text')
    for i in range(4, len(temp), 2):
        data_column_name.append(temp[i].text)
    temp = sp.find_all(class_ = 'bui-text cellheader selected-1vxxHvFg bui-text')
    for i in range(0, len(temp), 2):
        data_column_name.append(temp[i].text)

    click_expanded_button()
    sp = soup(driver.page_source, 'html.parser')
    temp = sp.find_all(class_ = 'bui-text cellheader bui-text')
    for i in range(4, len(temp), 2):
        data_column_name.append(temp[i].text)
        
    return data_column_name

In [6]:
# click standard button
def click_standard_button():
    time.sleep(3)
    driver.find_element('xpath', '//*[@id="stats-app-root"]/section/section/div[1]/div[2]/div/div[1]/div/div[1]/button').click()
    driver.find_element('xpath', '//*[@id="stats-app-root"]/section/section/div[1]/div[2]/div/div[1]/div/div[1]/button').click()
    time.sleep(3)
    sp = soup(driver.page_source, 'html.parser')

In [7]:
# click expanded button
def click_expanded_button():
    time.sleep(3)
    driver.find_element('xpath', '//*[@id="stats-app-root"]/section/section/div[1]/div[2]/div/div[1]/div/div[2]/button').click()
    driver.find_element('xpath', '//*[@id="stats-app-root"]/section/section/div[1]/div[2]/div/div[1]/div/div[2]/button').click()
    time.sleep(3)

In [8]:
# click next page button
def click_next_page_button():
    time.sleep(3)
    driver.find_element('xpath', '//*[@id="stats-app-root"]/section/section/div[3]/div[2]/div/div/div[2]/button').click()
    time.sleep(3)
    sp = soup(driver.page_source, 'html.parser')

In [9]:
# get standard data
def get_data():
    data = []
    
    click_standard_button()
    sp = soup(driver.page_source, 'html.parser')
    standard_data = []
    tt = sp.select('#stats-app-root tr td')
    for i in tt:
        standard_data.append(i.text)
    
    click_expanded_button()
    sp = soup(driver.page_source, 'html.parser')
    expanded_data = []
    tt = sp.select('#stats-app-root tr td')
    for i in tt:
        expanded_data.append(i.text)

    for i in range(len(standard_data)//17):
        data.extend(standard_data[i*17:i*17+17])
        data.extend(expanded_data[i*16+1:i*16+16])

    return data

```python
html = driver.get(url) 
# 有時候會 get 不完整，可以用這邊 reload 一次 XD
```

In [10]:
try:
    df_list = []
    while True:
        sp = soup(driver.page_source, 'html.parser')
        print('page start')
        time.sleep(2)
        name = get_player_name()
        time.sleep(2)
        position = get_player_position()
        time.sleep(2)
        current_data = get_data()
        time.sleep(2)
        for i in range(len(name)):
            row = []
            row.append(name[i])
            row.append(position[i])
            row.extend(current_data[i*32:i*32+32])
            df_list.append(row)
        click_next_page_button()
except:
    print('This is the last page :)')

page start
This is the last page :)


In [11]:
df = pd.DataFrame(df_list, columns=get_column_name())
df

,PLAYER,POSITION,TEAM,G,AB,R,H,2B,3B,HR,...,GO/AO,XBH,TB,IBB,BABIP,ISO,AB/HR,BB/K,BB%,SO%
0,Ketel Marte,2B,AZ,14,63,6,21,6,1,2,...,1.17,9,35,0,.422,.222,31.50,.125,.031,.246
1,Alek Thomas,CF,AZ,14,43,9,11,1,0,4,...,1.00,5,24,0,.241,.302,10.75,.364,.083,.229
2,Corbin Carroll,RF,AZ,14,54,10,16,1,1,2,...,1.38,4,25,1,.318,.167,27.00,.778,.109,.141
3,Geraldo Perdomo,SS,AZ,14,41,9,12,0,0,2,...,1.75,2,18,0,.370,.146,20.50,.500,.115,.231
4,Gabriel Moreno,C,AZ,14,52,4,13,1,0,4,...,1.30,5,26,0,.281,.250,13.00,.313,.088,.281
5,Tommy Pham,DH,AZ,13,51,10,14,2,0,3,...,2.50,5,25,0,.344,.216,17.00,.063,.019,.308
6,Pavin Smith,X,AZ,8,9,1,3,0,0,0,...,3.00,0,3,0,.429,.000,-.--,.500,.100,.200
7,Lourdes Gurriel,LF,AZ,14,56,4,15,3,0,2,...,1.31,5,24,1,.310,.161,28.00,.167,.034,.203
8,Christian Walker,1B,AZ,14,48,6,8,3,0,1,...,0.71,4,14,1,.226,.125,48.00,.706,.194,.274
9,Emmanuel Rivera,3B,AZ,10,13,2,3,0,0,0,...,1.00,0,3,0,.273,.000,-.--,.500,.071,.143


In [ ]:
df.to_csv('mlb_stats_test.csv', index=False, encoding='utf-8')

In [ ]:
driver.quit()